<a href="https://colab.research.google.com/github/fkvldjtm/LGMRepo/blob/master/ai08_sc42x_%EC%9D%B4%EA%B1%B4%EB%AF%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SC42x 
## 자연어처리 (Natural Language Processing)

# Part 1 : 개념 요약

> 다음의 키워드에 대해서 **한 줄**로 간단하게 요약해주세요. (세션 노트를 참고하여도 좋습니다.)<br/>
> **Tip : 아래 문제를 먼저 수행한 후 모델 학습 등 시간이 오래 걸리는 셀이 실행되는 동안 아래 내용을 작성하면 시간을 절약할 수 있습니다.**

**N421**
- Stopwords(불용어) : 문장에 많은 의미를 추가하지 않는 단어
- Stemming과 Lemmatization : Stemming은 어간을 추출하는 방법이고, Lemmatization은 단어들로부터 표제어를 찾아가는 방법
- Bag-of-Words : 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도에만 집중하는 텍스트 데이터 수치화 표현 방법
- TF-IDF : 단어의 빈도와 역 문서 빈도를 사용하여 DTM내의 각 단어들마다 중요한 정도를 가중치로 주는 방법

**N422**
- Word2Vec : 가장 많이 사용되는 Word Embedding 방법이며, 유사한 의미를 가진 단어는 유사한 벡터가 되는 특징이 있다.
- fastText : 단어를 벡터로 만드는 방법으로 하나의 단어 안에도 여러 단어들이 존재하는 것으로 간주하여 학습한다.

**N423**
- RNN : 과거의 정보를 사용하여 현재 및 미래의 입력에 대한 신경망의 성능을 개선하는 신경망 알고리즘
- LSTM, GRU : 기존의 RNN이 출력과 먼 위치에 있는 정보를 기억할 수 없다는 단점을 보완하여 장/단기 기억을 가능하게 설계한 신경망 구조
- Attention : 딥러닝 모델이 특정 벡터에 주목(attention)하게 만들어 모델의 성능을 높이는 기법

# Part 2 : Fake/Real News Dataset

한 주간 자연어처리 기법을 배우면서 여러분은 다양한 기술들을 접했습니다.<br/>
어떻게 텍스트 데이터를 다뤄야 하는지, 텍스트를 벡터화 하는 법, 문서에서 토픽을 모델하는 법 등 다양한 NLP 기법을 배웠는데요.<br/>
이번 스프린트 챌린지에선 [Fake/Real News Dataset](https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset)을 사용하여 배운 것들을 복습해보는 시간을 갖겠습니다.

**주의 : 모델의 성능을 최대한 끌어올리는 것이 아닌 모델 구동에 초점을 맞춰주세요.<br/>
모든 문제를 완료한 후에도 "시간이 남았다면" 정확도를 올리는 것에 도전하시는 것을 추천드립니다.**

In [2]:
# 코드 실행 전 seed를 지정하겠습니다.
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

## 2.0 데이터셋을 불러옵니다.

- 위 캐글 링크에서 데이터셋을 받아 업로드 합니다.<br/>
(직접 업로드하게 되면 시간이 꽤 걸리므로 **drive_mount** 나 **kaggle 연동**하시는 것을 추천드립니다.)

- 'label' 열을 만들어 Fake = 1, True = 0 로 레이블링해줍니다.
- 두 파일을 합쳐 하나의 데이터프레임에 저장해 준 후 데이터를 섞어줍니다.

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Fake.csv to Fake.csv
Saving True.csv to True.csv


In [4]:
from sklearn.utils import shuffle
import pandas as pd

In [5]:
fake_df = pd.read_csv('Fake.csv')
true_df = pd.read_csv('True.csv')

fake_df['label'] = 1
true_df['label'] = 0

df = pd.concat([fake_df, true_df])
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [6]:
df = shuffle(df)
df = df.reset_index(drop=True)
df.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",1
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",0
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",0
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",1
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",0


In [7]:
df = df[['text','label']]
df.head()

,text,label
0,"21st Century Wire says Ben Stein, reputable pr...",1
1,WASHINGTON (Reuters) - U.S. President Donald T...,0
2,(Reuters) - Puerto Rico Governor Ricardo Rosse...,0
3,"On Monday, Donald Trump once again embarrassed...",1
4,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",0


In [8]:
df.shape

(44898, 2)

## 2.1 TF-IDF 를 활용하여 특정 뉴스와 유사한 뉴스 검색하기

시간상 특별한 **전처리 없이** 아래 태스크를 수행하겠습니다.

### 2.1.1 TFidfVectorizer를 사용하여 문서-단어 행렬(Document-Term Matrix) 만들기

In [9]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
nlp = spacy.load("en_core_web_sm")

In [11]:
def tokenize(document):
    doc = nlp(document)
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True) and (token.is_alpha == True) and (token.pos_ != 'PRON')]

In [12]:
tfidf_vect = TfidfVectorizer(stop_words='english', tokenizer=tokenize, max_features=100)

In [13]:
dtm_tfidf = tfidf_vect.fit_transform(df['text'])

In [14]:
dtm_tfidf = pd.DataFrame(dtm_tfidf.todense(), columns=tfidf_vect.get_feature_names())
dtm_tfidf

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,accord,administration,america,american,ask,attack,campaign,candidate,case,change,city,clinton,come,country,court,day,deal,democratic,department,don,donald,election,federal,force,foreign,good,government,group,help,hillary,hold,house,image,include,issue,know,law,lead,leader,leave,...,police,policy,political,president,presidential,public,report,republican,republicans,reuters,right,russia,s,say,security,senate,state,statement,states,support,t,talk,tell,thing,think,time,trump,try,united,vote,want,washington,way,week,white,win,woman,work,world,year
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.447682,0.171354,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.378191,0.120410,0.192053,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.309016,0.090156,0.000000,0.0,0.144872,0.380453,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.374840,0.0,0.000000,0.000000,0.000000,0.168524,0.000000,0.000000,0.000000,0.000000,0.0,0.171204,0.000000,0.000000
1,0.000000,0.176992,0.043709,0.000000,0.000000,0.0,0.039009,0.000000,0.0,0.091275,0.0,0.000000,0.000000,0.000000,0.000000,0.037356,0.000000,0.00000,0.000000,0.00000,0.030186,0.038837,0.000000,0.044062,0.139271,0.000000,0.000000,0.040043,0.083089,0.000000,0.00000,0.251897,0.000000,0.035526,0.0,0.000000,0.000000,0.042373,0.000000,0.000000,...,0.0,0.224868,0.041223,0.183750,0.041868,0.000000,0.000000,0.000000,0.048303,0.027566,0.036347,0.103588,0.000000,0.275159,0.431767,0.0,0.031583,0.041470,0.039521,0.000000,0.000000,0.043951,0.030331,0.000000,0.000000,0.032608,0.354105,0.0,0.037121,0.000000,0.000000,0.073478,0.079183,0.000000,0.231936,0.000000,0.0,0.000000,0.044254,0.030511
2,0.000000,0.353134,0.000000,0.162675,0.159671,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.149064,0.000000,0.00000,0.192480,0.00000,0.000000,0.000000,0.188691,0.000000,0.000000,0.170099,0.141685,0.000000,0.331557,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.165139,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.110000,0.290076,0.000000,0.000000,0.548998,0.172292,0.0,0.000000,0.000000,0.000000,0.155654,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.108694,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,0.173688,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.142509,0.146306,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.126465,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.167780,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.219948,0.000000,0.000000,0.161425,0.000000,0.000000,0.000000,0.000000,0.216992,0.000000,0.494049,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.182579,0.000000,0.136612,0.570587,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.305684,0.091895,0.0,0.000000,0.0,0.087803,0.000000,0.068415,0.000000,0.000000,0.000000,0.08565,0.000000,0.00000,0.059137,0.000000,0.000000,0.000000,0.272843,0.000000,0.000000,0.000000,0.000000,0.087304,0.08338,0.070498,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.080533,0.084326,...,0.0,0.440535,0.000000,0.000000,0.164047,0.173026,0.000000,0.436381,0.000000,0.054004,0.071206,0.000000,0.000000,0.115513,0.000000,0.0,0.061873,0.000000,0.077425,0.000000,0.000000,0.000000,0.000000,0.000000,0.075500,0.191646,0.320179,0.0,0.072722,0.167422,0.000000,0.000000,0.000000,0.073333,0.075730,0.167035,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...

### 2.1.2 KNN 알고리즘을 사용하여 유사한 문서 검색하기

- **42번 인덱스의 문서**와 가장 유사한 **5개 문서(42번 포함)의 인덱스**와 **해당 인덱스의 레이블**을 나타내주세요.
- NN 모델의 파라미터 중 `algorithm = 'kd_tree'` 로 설정합니다.

In [15]:
from sklearn.neighbors import NearestNeighbors

In [16]:
nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm_tfidf)

NearestNeighbors(algorithm='kd_tree')

In [17]:
nn.kneighbors([dtm_tfidf.iloc[42]])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  "X does not have valid feature names, but"


(array([[0.        , 0.        , 0.76294471, 0.76294471, 0.79613619]]),
 array([[   42, 33954, 24159, 29809, 39767]]))

In [18]:
print('Label :',df['label'][42],', Text : ', df['text'][42][:100])
print('Label :',df['label'][24159],', Text : ', df['text'][24159][:100])
print('Label :',df['label'][29809],', Text : ', df['text'][29809][:100])
print('Label :',df['label'][33954],', Text : ', df['text'][33954][:100])
print('Label :',df['label'][39767],', Text : ', df['text'][39767][:100])

Label : 1 , Text :  A 70-year tradition at East Liverpool High School was removed from the graduation ceremony, but stud
Label : 1 , Text :   These are people who claim to love diversity, and they don t love diversity   they just want people
Label : 1 , Text :   These are people who claim to love diversity, and they don t love diversity   they just want people
Label : 1 , Text :  A 70-year tradition at East Liverpool High School was removed from the graduation ceremony, but stud
Label : 1 , Text :  Pastor Steven Anderson of Faithful Word Baptist Church in Tempe, Arizona, is a bigot in the truest s


## 2.2 Keras Embedding을 사용하여 분류하기

### 2.2.0 데이터셋 split

- Train, Test 데이터셋으로 분리(Split)하여 주세요.

In [19]:
from sklearn.model_selection import train_test_split

df.label = df.label.astype(int)

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42 , stratify=df['label'])
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((35918,), (8980,), (35918,), (8980,))

### 2.2.1 단어 벡터의 평균을 이용하여 분류해보기

N422에서 했던 단어 임베딩 벡터의 평균을 사용하여 문장을 분류하는 작업을 수행해봅시다.<br/>
인스턴스마다 텍스트 길이가 길고 시간이 오래 걸리므로 시간상 epoch 수를 **10 이하**로 하는 것을 추천드립니다.<br/>
모델 구동이 목적이므로 임베딩 차원 수를 크지 않게(50이하)로 설정해주세요.<br/>
**권장사항 : `max_len` 은 텍스트 길이 평균보다 높게 설정해주세요.**<br/>

> **Tip : 모델이 학습하는 동안 2.2.3의 내용을 작성하면 시간을 절약할 수 있습니다.**


In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import gensim.downloader as api
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

In [21]:
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [22]:
# Tokenizer
tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts(X_train) # 단어 사전 형성

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [23]:
print(f'Mean length of train set: {np.mean([len(sent) for sent in X_train], dtype=int)}')

Mean length of train set: 298


In [24]:
max_len = 500

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

y_train=np.array(y_train)
y_test=np.array(y_test)

In [25]:
X_train.shape, y_train.shape

((35918, 500), (35918,))

In [26]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

125797

In [27]:
from collections import Counter

embedding_matrix = np.zeros((vocab_size, 300))

def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None

oov = Counter()
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp
    else:
        oov[word] += 1

In [28]:
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))

In [29]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
449/449 [==============================] - 11s 24ms/step - loss: 0.6577 - acc: 0.6293 - val_loss: 0.6259 - val_acc: 0.8004
Epoch 2/10
449/449 [==============================] - 10s 23ms/step - loss: 0.5991 - acc: 0.8143 - val_loss: 0.5759 - val_acc: 0.8352
Epoch 3/10
449/449 [==============================] - 11s 23ms/step - loss: 0.5540 - acc: 0.8345 - val_loss: 0.5366 - val_acc: 0.8360
Epoch 4/10
449/449 [==============================] - 11s 23ms/step - loss: 0.5183 - acc: 0.8404 - val_loss: 0.5051 - val_acc: 0.8434
Epoch 5/10
449/449 [==============================] - 11s 24ms/step - loss: 0.4893 - acc: 0.8467 - val_loss: 0.4788 - val_acc: 0.8505
Epoch 6/10
449/449 [==============================] - 10s 23ms/step - loss: 0.4651 - acc: 0.8526 - val_loss: 0.4568 - val_acc: 0.8580
Epoch 7/10
449/449 [==============================] - 10s 23ms/step - loss: 0.4447 - acc: 0.8586 - val_loss: 0.4382 - val_acc: 0.8612
Epoch 8/10
449/449 [==============================] - 10s 23ms

In [30]:
loss, acc = model.evaluate(X_test, y_test)
print("loss = {}, acc = {}]".format(loss,acc))

281/281 [==============================] - 3s 9ms/step - loss: 0.3999 - acc: 0.8692
loss = 0.39988359808921814, acc = 0.8691536784172058]


### 2.2.2 LSTM을 사용하여 텍스트 분류 수행해보기

N423에서 했던 단어 임베딩 벡터를 사용하여 문장을 분류하는 작업을 수행해봅시다.<br/>
인스턴스마다 텍스트 길이가 길어 시간이 매우 오래 걸리므로 <br/>
**층을 최소한으로 쌓고**, epoch 수를 **3 이하**로 하는 것을 추천드립니다.<br/>

> **Tip : 모델이 학습하는 동안 2.2.3의 내용을 작성하면 시간을 절약할 수 있습니다.**


In [31]:
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop

In [32]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(vocab_size,300,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='hl1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)

    return model

In [33]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 500)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 500, 300)          37739100  
                                                                 
 lstm (LSTM)                 (None, 64)                93440     
                                                                 
 hl1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [34]:
model.fit(X_train, y_train, batch_size=128, epochs=5, validation_split=0.2)

Epoch 1/5
225/225 [==============================] - 376s 2s/step - loss: 0.1204 - accuracy: 0.9591 - val_loss: 0.0397 - val_accuracy: 0.9891
Epoch 2/5
225/225 [==============================] - 386s 2s/step - loss: 0.0492 - accuracy: 0.9837 - val_loss: 0.0293 - val_accuracy: 0.9908
Epoch 3/5
225/225 [==============================] - 375s 2s/step - loss: 0.0372 - accuracy: 0.9889 - val_loss: 0.0351 - val_accuracy: 0.9875
Epoch 4/5
225/225 [==============================] - 371s 2s/step - loss: 0.0256 - accuracy: 0.9927 - val_loss: 0.0260 - val_accuracy: 0.9922
Epoch 5/5
225/225 [==============================] - 369s 2s/step - loss: 0.0219 - accuracy: 0.9938 - val_loss: 0.0247 - val_accuracy: 0.9936


In [35]:
loss, acc = model.evaluate(X_test, y_test)
print("loss = {}, acc = {}]".format(loss,acc))

281/281 [==============================] - 29s 104ms/step - loss: 0.0188 - accuracy: 0.9949
loss = 0.018755799159407616, acc = 0.9948775172233582]


### 2.2.3 위에서 실행한 내용에 대해 다시 알아봅시다.

#### a) 데이터셋을 학습할 때 사용하는 `pad_sequences`  메서드에 대해 설명해주세요.<br/>어떤 기능을 하나요? 모델을 학습할 때 왜 필요한가요?

가변 길이 시퀸스 예측 문제의 경우 각 시퀸스의 길이가 같도록 데이터를 변형해야 한다. 이때, pad_sequences가 사용된다.
이 벡터화는 코드가 선택한 딥러닝 알고리즘에 대해 일괄적으로 행렬 연산을 수행할 수 있게 도와준다.
다시 말해 병렬 연산을 위해서 여러 문장의 길이를 임의로 동일하게 맞춰주는 작업이 필요할 때 사용한다.

#### b) 2.2.1과 2.2.2에서 사용한 각 모델의 evaluation 성능은 어떻게 나왔나요?<br/>각 모델의 장단점은 무엇이라고 생각하나요?

2.2.1의 결과 (Word2Vec)

loss: 0.3999 - acc: 0.8692

2.2.2의 결과 (LSTM)

loss: 0.0157 - accuracy: 0.9958

적은 epoch만으로도 좋은 성능을 보여준다.

Word2Vec

장점: 대량의 학습 데이터셋을 아주 빠르게 단어의 벡터값을 학습할 수 있다.

단점: 말뭉치에 등장하지 않은 단어에 대해서는 임베딩 벡터를 만들지 못한다.

LSTM

장점: 다양한 다른 업무에 적용 가능하다. 각각의 메모리와 결과값이 컨트롤이 가능하다. 성능이 좋다.

단점: 장기 의존성 문제가 개선되었어도 문장이 길어지면 문제가 된다. 속도가 느리다.



#### c) 종래의 RNN(Recurrent Neural Networks) 대신 LSTM(Long-Short Term Memory)을 사용하는 이유는 무엇인가요?<br/>(i.e. RNN에 비해 LSTM의 좋은 점을 설명해주세요.)

일반적인 RNN의 경우 짧은 시퀸스를 처리할 경우 유리하며, 관련 정보와 그 정보를 사용하는 지점 사이 거리가 멀어지는 경우 학습 능력이 현저하게 저하된다.
이 문제의 원인은 데이터의 시퀸스가 길어지면 RNN의 역전파 알고리즘인 BPTT(Backpropagation Through Time)가 장기 의존성 학습에 어려움이 있기 때문인데 가중치를 업데이트 하는 과정에서 1보다 작은 값들이 계속 곱해지면서 기울기가 사라지는 기울기 소실 문제가 발생한다.

이 장기 의존성 문제를 해결하기 위해 나온것이 LSTM(Long Short-Term Memory) 이다. 이 모델은 훈련이 빠르게 수렴되고, 장기간 의존성을 감지한다. 데이터를 계산하는 위치에 Input, Forget, Output 게이트 3가지가 추가되어 각 상태값을 메모리 공간 셀에 저장하고, 데이터를 접하는 게이트 부분을 조정하여 불필요한 연산, 오차 등을 줄여 장기 의존성 문제를 일정 부분 해결한다. 

#### d) LSTM이나 RNN을 사용하는 예시를 **3개**이상 제시하고 해당되는 경우에 왜 LSTM이나 RNN을 사용하는 것 적절한지 간단하게 설명해주세요.

번역, 챗봇, 주가 예측 등에 사용된다.

RNN은 기본 신경망의 변형된 형태로 이전 결과값을 다시 Input으로 사용해서 다음 결과값을 얻는다. 즉, 이전 결과값이 다음 결과값에 영향을 미친다. 즉, RNN은 데이터 시퀸스 패턴을 학습하는 것이다. 이는 시간 흐름이나 연속된 값의 패턴을 찾는데 용이하다.

RNN은 다음에 나올 값을 예측하는 것이므로, 예를 들어 문장의 첫 단어를 제시하면 다음에 어떤 단어가 나오는 것이 가장 확률이 높인지 예측하는 모델을 만들 수 있다. 이렇게 RNN을 이용하면 번역, 챗봇, 주가 예측 등 시퀸스 기반의 다양한 모델을 만들 수 있다.

#### e) 이외에 N424 에서 배운 자연어처리 모델과 관련된 키워드를 3개 이상 적어주세요. <br/> (해당 키워드에 대한 설명은 옵션입니다.)

1. Transformer : RNN 기반 모델이 가진 구조적 단점은 단어가 순차적으로 들어오기 때문에 처리해야 하는 시퀸스가 길수록 연산 시간이 길어진다. 이를 해결하기 위해 나온 모델로 Attention 메커니즘을 극대화한 모델이다. Input sentence를 널어 Ouput sentence를 생성해낸다.
2. Positional Encoding : Transformer는 단어 입력을 순차적으로 받는 방식이 아니므로 단어의 위치 정보를 다른 방식으로 알려줄 필요가 있다.(병렬화를 위해 모든 단어 벡터를 동시에 입력 받음) Transformer는 단어의 위치 정보를 얻기 위해서 각 단어의 임베딩 벡터에 위치 정보들을 더하여 모델의 입력으로 사용하는데 이것을 Positional Encoding이라고 한다.
3. Self-attention : 문장 내부 요소의 관계를 잘 파악하기 위해서, 문장 자신에 대해 어텐션 메커니즘을 적용하는 것
4. Multi-Head Attention : Self-Attention을 동시에 병렬적으로 실행하는 것

# Advanced Goals: 3점을 획득하기 위해선 아래의 조건 중 하나 이상을 만족해야합니다
 
- 2.1 에서 TF-IDF(`TfidfVectorizer`)가 아닌 방법을 사용하여 유사도 검색을 수행해보세요.<br/>
TF-IDF와 해당 방법의 차이를 설명해주세요. 
- 2.2 에서 사용한 방법을 재사용하되 하이퍼 파라미터를 조정하거나 모델 구조를 변경하여 성능을 올려봅시다.<br/>**(주의 : GridSearch, RandomSearch 등의 방법을 사용하여도 좋으나 시간이 오래 걸리므로 범위를 잘 선택해야 합니다.)**

In [36]:
# !pip install keras-tuner

In [37]:
import kerastuner as kt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

ModuleNotFoundError: ignored

In [ ]:
def model_builder(rate):

  # model = Sequential()
  # model.add(Embedding(vocab_size, 300, max_len))
  # model.add(LSTM(64))
  # model.add(Dense(nodes, activation='relu'))
  # model.add(Dropout(rate))
  # model.add(Dense(1, activation='sigmoid'))
  # model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

  model = Sequential()
  model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False))
  model.add(GlobalAveragePooling1D())
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate = rate), 
                metrics=['acc'])
  return model

In [ ]:
rate = [1e-2, 1e-3, 1e-4]
epochs = [10, 20, 30]
batch_size=[64, 128, 256]

# param_grid = dict(batch_size=batch_size, validation_split=validation_split, epochs=epochs, nodes=nodes, rate=rate)
param_grid = dict(batch_size=batch_size, epochs=epochs, rate=rate)


In [ ]:
model = KerasClassifier(build_fn=model_builder, verbose=0)

In [ ]:
# GridSearch CV를 만들기
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1)
grid_result = grid.fit(X_train, y_train)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [ ]:
# model = KerasClassifier(build_fn=model_builder,batch_size=128,validation_split=0.2, verbose=0)

In [ ]:
# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, error_score=0)
# grid_result = grid.fit(X_train, y_train)